Importing libraries

In [8]:
import json as js
from collections import defaultdict
import string
from keras import layers #type : ignore
import nltk
from tqdm import tqdm

loading paths

In [9]:
with open('Config_RNN.json','r') as file:
    paths = js.load(file)

Loading initial train data

In [10]:
with open(paths["Caption_Train"],'r') as file:
    train_data = js.load(file)

Extracting imp data

In [11]:
newdict = {}
newdict["img_data"] = train_data["images"]
newdict["annotations_data"] = train_data["annotations"]

Creating Corpus

In [12]:
img_data = newdict["img_data"]
annotations_data = newdict["annotations_data"]
captions_dict = defaultdict(list)

for ann in annotations_data:
    captions_dict[ann["image_id"]].append(ann["caption"])

image_caption_data = {}
for img in img_data:
    file_id = img["file_name"]
    img_id = img["id"]
    image_caption_data[file_id] = captions_dict[img_id]

with open(paths["Corpus"], "w") as f:
    js.dump(image_caption_data, f, indent=4)


loading corpus

In [13]:
with open(paths["Corpus"], "r") as f:
    Corpus_data = js.load(f)

Loading nltk dependency

In [14]:
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

Text Tokenization

In [17]:
pos_tagged_captions = {}
noun_captions = {}

for item in tqdm(Corpus_data):
    tagged = []
    for caption in item:
        tokens = nltk.word_tokenize(caption)
        pos_tags = nltk.pos_tag(tokens)
        tagged.append(pos_tags)
    pos_tagged_captions[item['file_name']] = tagged

for image_id, tagged_captions in tqdm(pos_tagged_captions.items()):
    noun_lists = []
    for tagged in tagged_captions:
        nouns = [word for word, tag in tagged if tag in ('NN', 'NNS', 'NNP', 'NNPS')]
        noun_lists.append(nouns)
    noun_captions[image_id] = noun_lists

  0%|          | 0/118287 [00:07<?, ?it/s]


LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger_eng[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger_eng/[0m

  Searched in:
    - 'C:\\Users\\Dell/nltk_data'
    - 'c:\\Users\\Dell\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data'
    - 'c:\\Users\\Dell\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data'
    - 'c:\\Users\\Dell\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data'
    - 'C:\\Users\\Dell\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


storing noune tags

In [ ]:
with open(paths['Nouned_Corpus'],'w') as f:
    js.dump(noun_captions,f)

In [ ]:
with open(paths["Corpus"], 'r') as f1:
    captions_data = js.load(f1)

with open(paths['Nouned_Corpus'], 'r') as f2:
    features_data = js.load(f2)

dataset = []

for image_id,nouns in tqdm(features_data.items()): 
    noun_caption = zip(nouns, captions_data[image_id])
    for noun, caption in noun_caption:
        dataset.append({
            "input": " ".join(noun),
            "output": caption
        })

with open(paths["Preprocessed_data"], "w") as out_file:
    js.dump(dataset, out_file, indent=2)

100%|██████████| 118287/118287 [00:00<00:00, 379465.09it/s]


Padding

In [ ]:
with open(paths["Preprocessed_data"], 'r') as f:
    data = js.load(f)

max_length = max(len(item['input']) for item in data)

for item in data:
    item['input'] = item['input'].ljust(max_length)

with open(paths["Padded_preprocessed_data"], 'w') as f:
    js.dump(data, f, indent=4)

print("All captions padded to length:", max_length)

All captions padded to length: 158
